## Imports

In [123]:
pip install --upgrade scikit-learn

Requirement already up-to-date: scikit-learn in c:\anaconda\lib\site-packages (0.24.2)Note: you may need to restart the kernel to use updated packages.



In [5]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.1.3
Sklearn  0.24.2


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [6]:
CLOUD = False

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "your_kaggle_username"
    os.environ['KAGGLE_KEY']      = "your_kaggle_api_key"  # See https://www.kaggle.com/docs/api
    !pip install --upgrade kaggle
    !kaggle competitions download -c titanic
    DATA_PATH = "./"

else:
    DATA_PATH = "../../Datasets/Tabular/titanic/"

## Load data

In [7]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


In [8]:
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Check missings

In [9]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [10]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [11]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)

get_Title_from_Name = lambda name: name.split(',')[1].split('.')[0].strip()

# YOUR CODE HERE
df['Title']      = df['Name'].map(get_Title_from_Name)
df_test['Title'] = df_test['Name'].map(get_Title_from_Name)

df['Title'].unique(),df_test['Title'].unique

(array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
        'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
        'Jonkheer'], dtype=object),
 <bound method Series.unique of PassengerId
 892         Mr
 893        Mrs
 894         Mr
 895         Mr
 896        Mrs
          ...  
 1305        Mr
 1306      Dona
 1307        Mr
 1308        Mr
 1309    Master
 Name: Title, Length: 418, dtype: object>)

In [12]:
assert df['Title'].values[0] == "Mr"
assert df['Title'].values[1] == "Mrs"
assert df['Title'].values[2] == "Miss"

assert df_test['Title'].values[0] == "Mr"
assert df_test['Title'].values[1] == "Mrs"
assert df_test['Title'].values[414] == "Dona"

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [13]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [14]:
# Use map to apply the prevous dict

df["Title"] = df.Title.map(title_dictionary) 
df_test["Title"] =df_test.Title.map(title_dictionary)

# YOUR CODE HERE
#raise NotImplementedError()

In [15]:
assert df['Title'].values[886] == "Officer"
assert df_test['Title'].values[417] == "Master"

# Exercise OPTINAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [16]:
#df_test['Ticket']

print("There are {} unique Ticket values (training data).".format(len(df.Ticket.unique())))
print("There are {} unique Ticket values (testing data).".format(len(df_test.Ticket.unique())))

There are 681 unique Ticket values (training data).
There are 363 unique Ticket values (testing data).


# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [17]:
#df_test['Cabin']

cabin_test= df_test.Cabin.isnull().sum()
cabin_unique_test=df_test.Cabin.value_counts()
print(f"There are {cabin_test} missing values in Cabin df_test")
print(f"There are {len(cabin_unique_test)} unique values  in Cabin df_test")
print('\n')
cabin_train=df.Cabin.isnull().sum()
cabin_unique_train=df.Cabin.value_counts()
print(f"There are {cabin_train} missing values in Cabin df")
print(f"There are {len(cabin_unique_train)} unique values  in Cabin df")


There are 327 missing values in Cabin df_test
There are 76 unique values  in Cabin df_test


There are 687 missing values in Cabin df
There are 147 unique values  in Cabin df


# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [18]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin']) # # X_TEST DATA (NEW DATA)

In [19]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [53]:

num_4_treeModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value=-9999)),
])

cat_4_treeModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
    ('ordinal', preprocessing.OrdinalEncoder(handle_unknown='error'))
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
    ('cat', cat_4_treeModels, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

# YOUR CODE HERE
#raise NotImplementedError()


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value=-9999,
                                                                strategy='constant'))]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('ordinal',
                                                  OrdinalEncoder())]),
                                 ['Sex', 'Embarked', 'Title'])])

In [30]:
assert type(tree_prepro)      is compose._column_transformer.ColumnTransformer
assert type(num_4_treeModels) is pipeline.Pipeline
assert type(cat_4_treeModels) is pipeline.Pipeline
assert len(num_4_treeModels) == 1
assert len(cat_4_treeModels) == 2

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [62]:
pip install catboost

In [54]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [55]:

tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":   ExtraTreesClassifier(n_estimators=100),
  "Random Forest": RandomForestClassifier(n_estimators=100),
  "AdaBoost": AdaBoostClassifier(n_estimators=100),
  "Skl GBM": GradientBoostingClassifier(n_estimators=100),
    
  "Skl HistGBM":   HistGradientBoostingClassifier(max_iter=100),
  "XGBoost": XGBClassifier(n_estimators=100),
  "LightGBM": LGBMClassifier(n_estimators=100),
  "CatBoost": CatBoostClassifier(n_estimators=100)
}
tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}
tree_classifiers["LightGBM"]



# YOUR CODE HERE


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=-9999,
                                                                                 strategy='constant'))]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder())]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [56]:
for pipe in tree_classifiers.values():
    assert type(pipe) is pipeline.Pipeline

In [27]:
print()

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [57]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(x, y, test_size=0.2 ,stratify = y, random_state=4)




# YOUR CODE HERE
print(type(x_train))
print(type(y_val))

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})


for model_name, model in tree_classifiers.items():
    start_time = time.time()
    
    
    
    model.fit(x_train,y_train)
    
    # FOR EVERY PIPELINE (PREPRO + MODEL) -> TRAIN WITH TRAIN DATA (x_train)
    pred = model.predict(x_val)
    
    # GET PREDICTIONS USING x_val
    # CODE HERE

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              
# YOUR CODE HERE


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


C:\Anaconda\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:21:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.073611
0:	learn: 0.6694333	total: 773us	remaining: 76.6ms
1:	learn: 0.6484799	total: 2.55ms	remaining: 125ms
2:	learn: 0.6300092	total: 3.21ms	remaining: 104ms
3:	learn: 0.6081509	total: 4.78ms	remaining: 115ms
4:	learn: 0.5925861	total: 6.43ms	remaining: 122ms
5:	learn: 0.5754528	total: 8.08ms	remaining: 127ms
6:	learn: 0.5606121	total: 9.76ms	remaining: 130ms
7:	learn: 0.5481592	total: 11.4ms	remaining: 131ms
8:	learn: 0.5352224	total: 12.7ms	remaining: 128ms
9:	learn: 0.5240170	total: 14.3ms	remaining: 128ms
10:	learn: 0.5144419	total: 16ms	remaining: 129ms
11:	learn: 0.5054878	total: 17.7ms	remaining: 130ms
12:	learn: 0.4958095	total: 19.4ms	remaining: 13

,Model,Accuracy,Bal Acc.,Time
1,XGBoost,85.474860,84.670619,0.187487
2,Skl HistGBM,84.916201,83.675889,0.846682
3,LightGBM,84.916201,83.675889,0.135853
4,CatBoost,84.916201,83.405797,0.362830
5,AdaBoost,84.357542,84.031621,0.193872
6,Skl GBM,84.357542,82.951252,0.120948
7,Decision Tree,81.564246,80.948617,0.019985
8,Random Forest,81.005587,78.873518,0.209897
9,Extra Trees,79.888268,78.774704,0.212879


In [34]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [58]:
from sklearn.model_selection import cross_val_predict

skf = model_selection.StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
# YOUR CODE HERE


results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})


for model_name, model in tree_classifiers.items():
    start_time = time.time()


        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
    pred = cross_val_predict(model, x,y, cv=skf)

    total_time = time.time() - start_time

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

C:\Anaconda\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:21:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Anaconda\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:21:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Anaconda\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

74:	learn: 0.3710825	total: 140ms	remaining: 46.8ms
75:	learn: 0.3701726	total: 143ms	remaining: 45ms
76:	learn: 0.3695566	total: 145ms	remaining: 43.2ms
77:	learn: 0.3691345	total: 147ms	remaining: 41.3ms
78:	learn: 0.3682430	total: 149ms	remaining: 39.5ms
79:	learn: 0.3675307	total: 150ms	remaining: 37.6ms
80:	learn: 0.3665475	total: 152ms	remaining: 35.7ms
81:	learn: 0.3659918	total: 154ms	remaining: 33.8ms
82:	learn: 0.3650207	total: 156ms	remaining: 32ms
83:	learn: 0.3649652	total: 157ms	remaining: 29.9ms
84:	learn: 0.3643130	total: 159ms	remaining: 28.1ms
85:	learn: 0.3637823	total: 161ms	remaining: 26.2ms
86:	learn: 0.3633456	total: 163ms	remaining: 24.4ms
87:	learn: 0.3629597	total: 165ms	remaining: 22.5ms
88:	learn: 0.3624419	total: 167ms	remaining: 20.6ms
89:	learn: 0.3618074	total: 169ms	remaining: 18.7ms
90:	learn: 0.3614405	total: 170ms	remaining: 16.8ms
91:	learn: 0.3609055	total: 172ms	remaining: 15ms
92:	learn: 0.3601645	total: 174ms	remaining: 13.1ms
93:	learn: 0.35994

78:	learn: 0.3705426	total: 149ms	remaining: 39.6ms
79:	learn: 0.3699374	total: 151ms	remaining: 37.8ms
80:	learn: 0.3694046	total: 153ms	remaining: 35.9ms
81:	learn: 0.3686348	total: 155ms	remaining: 34ms
82:	learn: 0.3683184	total: 157ms	remaining: 32.1ms
83:	learn: 0.3678292	total: 159ms	remaining: 30.2ms
84:	learn: 0.3673124	total: 161ms	remaining: 28.3ms
85:	learn: 0.3669754	total: 162ms	remaining: 26.4ms
86:	learn: 0.3666004	total: 164ms	remaining: 24.5ms
87:	learn: 0.3663965	total: 165ms	remaining: 22.5ms
88:	learn: 0.3660126	total: 167ms	remaining: 20.7ms
89:	learn: 0.3659260	total: 169ms	remaining: 18.8ms
90:	learn: 0.3656432	total: 171ms	remaining: 16.9ms
91:	learn: 0.3644602	total: 174ms	remaining: 15.1ms
92:	learn: 0.3636887	total: 176ms	remaining: 13.2ms
93:	learn: 0.3631147	total: 177ms	remaining: 11.3ms
94:	learn: 0.3624777	total: 180ms	remaining: 9.46ms
95:	learn: 0.3620041	total: 181ms	remaining: 7.56ms
96:	learn: 0.3616828	total: 183ms	remaining: 5.67ms
97:	learn: 0.3

69:	learn: 0.3782361	total: 136ms	remaining: 58.4ms
70:	learn: 0.3776208	total: 139ms	remaining: 56.8ms
71:	learn: 0.3771620	total: 142ms	remaining: 55.2ms
72:	learn: 0.3763860	total: 144ms	remaining: 53.4ms
73:	learn: 0.3759787	total: 147ms	remaining: 51.5ms
74:	learn: 0.3751937	total: 148ms	remaining: 49.5ms
75:	learn: 0.3747823	total: 150ms	remaining: 47.5ms
76:	learn: 0.3739244	total: 155ms	remaining: 46.2ms
77:	learn: 0.3735051	total: 157ms	remaining: 44.3ms
78:	learn: 0.3731327	total: 158ms	remaining: 42.1ms
79:	learn: 0.3725556	total: 161ms	remaining: 40.2ms
80:	learn: 0.3720531	total: 163ms	remaining: 38.2ms
81:	learn: 0.3717355	total: 165ms	remaining: 36.2ms
82:	learn: 0.3715421	total: 168ms	remaining: 34.5ms
83:	learn: 0.3713385	total: 170ms	remaining: 32.4ms
84:	learn: 0.3708003	total: 172ms	remaining: 30.4ms
85:	learn: 0.3702888	total: 174ms	remaining: 28.3ms
86:	learn: 0.3699254	total: 176ms	remaining: 26.3ms
87:	learn: 0.3693713	total: 178ms	remaining: 24.2ms
88:	learn: 0

65:	learn: 0.3681883	total: 137ms	remaining: 70.5ms
66:	learn: 0.3676959	total: 139ms	remaining: 68.4ms
67:	learn: 0.3673144	total: 141ms	remaining: 66.3ms
68:	learn: 0.3666364	total: 143ms	remaining: 64.1ms
69:	learn: 0.3664939	total: 143ms	remaining: 61.5ms
70:	learn: 0.3655386	total: 145ms	remaining: 59.3ms
71:	learn: 0.3644949	total: 147ms	remaining: 57.1ms
72:	learn: 0.3638552	total: 150ms	remaining: 55.5ms
73:	learn: 0.3633266	total: 152ms	remaining: 53.2ms
74:	learn: 0.3627934	total: 154ms	remaining: 51.2ms
75:	learn: 0.3622268	total: 156ms	remaining: 49.2ms
76:	learn: 0.3615621	total: 158ms	remaining: 47.1ms
77:	learn: 0.3611951	total: 161ms	remaining: 45.4ms
78:	learn: 0.3609441	total: 163ms	remaining: 43.3ms
79:	learn: 0.3598569	total: 165ms	remaining: 41.2ms
80:	learn: 0.3591100	total: 167ms	remaining: 39.2ms
81:	learn: 0.3582524	total: 169ms	remaining: 37.1ms
82:	learn: 0.3570672	total: 171ms	remaining: 35.1ms
83:	learn: 0.3565232	total: 174ms	remaining: 33ms
84:	learn: 0.3

,Model,Accuracy,Bal Acc.,Time
1,Skl GBM,83.613917,81.631675,1.168576
2,Skl HistGBM,83.052750,81.396798,8.952123
3,CatBoost,82.828283,80.332662,4.964041
4,LightGBM,82.491582,80.996549,1.617574
5,XGBoost,81.930415,80.375803,2.415546
6,AdaBoost,81.593715,80.543572,1.951406
7,Random Forest,80.920314,79.390758,2.015453
8,Extra Trees,80.246914,78.844310,2.041137
9,Decision Tree,79.797980,78.424887,0.353797


In [59]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 7.1
Train with all data the best model

In [60]:
# best_model = tree_classifiers["SELECT MY BEST MODEL HERE"]

# Fit best model with all data


# YOUR CODE HERE
best_model =   tree_classifiers["LightGBM"] 
best_model.fit(x,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=-9999,
                                                                                 strategy='constant'))]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('ordinal',
                                                                   OrdinalEncoder())]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('lgbmclassifier', LGBMClassifier())])

# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [61]:
# test_pred = # Get the predictions for x_test

# YOUR CODE HERE
test_pred = best_model.predict(x_test)

ValueError: Found unknown categories ['missing'] in column 2 during transform

In [48]:
assert len(test_pred) == 418
assert np.unique(test_pred).tolist() == [0,1]

NameError: name 'test_pred' is not defined

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [62]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

NameError: name 'test_pred' is not defined

In [50]:
sub.to_csv("sub.csv")

NameError: name 'sub' is not defined

In [51]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"

Traceback (most recent call last):
  File "c:\anaconda\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\anaconda\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda\Scripts\kaggle.exe\__main__.py", line 4, in <module>
  File "c:\anaconda\lib\site-packages\kaggle\__init__.py", line 23, in <module>
    api.authenticate()
  File "c:\anaconda\lib\site-packages\kaggle\api\kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in C:\Users\Farrukh Bulbulov\.kaggle. Or use the environment method.
